In [1]:
import os
import h5py
from astropy.table import Table, Column
import numpy as np
import pandas as pd

def consolidate_hdf5_files_with_centrals(input_folder, output_file, redmapper_centrals_data):
    # Open the main consolidated HDF5 file
    with h5py.File(output_file, 'w') as main_file:
        
        # Iterate over individual .h5 files in the input folder
        for filename in os.listdir(input_folder):
            if filename.endswith('.h5'):
                cluster_id = filename.split('.')[0]  # Extract cluster ID from filename
                
                # Open individual HDF5 file and read its content
                with h5py.File(os.path.join(input_folder, filename), 'r') as cluster_file:
                    # Create a group in the main file for this cluster
                    cluster_group = main_file.create_group(cluster_id)
                    
                    # Add 'centrals' data for the cluster
                    centrals_group = cluster_group.create_group('centrals')
                    mask = redmapper_centrals_data['mem_match_id'] == int(cluster_id)
                    for col_name in redmapper_centrals_data.colnames:
                        centrals_group.create_dataset(col_name, data=redmapper_centrals_data[col_name][mask])
                    
                    # Copy data from the individual file to the main file
                    for category in cluster_file.keys():
                        if category != 'centrals':  # We've already handled 'centrals'
                            category_group = cluster_group.create_group(category)
                            for halo_id in cluster_file[category].keys():
                                halo_group = category_group.create_group(halo_id)
                                for dataset in cluster_file[category][halo_id].keys():
                                    data = cluster_file[category][halo_id][dataset][:]
                                    halo_group.create_dataset(dataset, data=data)
                                for attr in cluster_file[category][halo_id].attrs.keys():
                                    halo_group.attrs[attr] = cluster_file[category][halo_id].attrs[attr]

    print("Consolidation complete!")

base_path = '/lustre/work/client/users/shuleic/Cardinalv3/'
# suffix = '_lgt20'
suffix = '_lgt05'
redmapper_member_data_old = Table.read(os.path.join(base_path, f'chunhao-redmapper{suffix}_mem_data_all.fits'))
redmapper_member_data = Table.read(os.path.join(base_path, f'chunhao-redmapper{suffix}_mem_data_all_new.fits'))
redmapper_members = Table.read(os.path.join(base_path, 'redmapper_v4_v8_v51_y6_v7/run/', f'Cardinal-3Y6a_v2.0_run_run_redmapper_v0.8.1{suffix}_vl02_catalog_members.fit'))
redmapper_data = Table.read(os.path.join(base_path, 'redmapper_v4_v8_v51_y6_v7/run/', f'Cardinal-3Y6a_v2.0_run_run_redmapper_v0.8.1{suffix}_vl02_catalog.fit'))

output_file = os.path.join(base_path, f'sorted-chunhao-redmapper{suffix}_data_new.h5')


# Get redMaPPer central members

In [25]:
import pandas as pd
from astropy.table import Table

# Convert redmapper_member_data to a pandas DataFrame
redmapper_member_df = redmapper_member_data.to_pandas()

# Create a list to hold expanded data for redmapper_data with an additional index
redmapper_data_expanded = []

for mem_id, ids in zip(redmapper_data['mem_match_id'], redmapper_data['id_cent']):
    for id_cent in ids:
        redmapper_data_expanded.append({'mem_match_id': mem_id, 'id_cent': id_cent})

# Create a DataFrame for the expanded redmapper_data
redmapper_data_df = pd.DataFrame(redmapper_data_expanded)

# Add an index to preserve the original order
redmapper_data_df['original_index'] = redmapper_data_df.index

# Merge the DataFrames on mem_match_id and id_cent
central_members_df = pd.merge(redmapper_member_df, redmapper_data_df, 
                               left_on=['mem_match_id', 'coadd_object_id'], 
                               right_on=['mem_match_id', 'id_cent'], 
                               how='inner')

# Sort by original index to preserve the order of id_cent from each mem_match_id
central_members_df = central_members_df.sort_values(by='original_index')

# Convert back to an Astropy Table
central_members = Table.from_pandas(central_members_df)
redmapper_centrals_path = os.path.join(base_path, f'chunhao-redmapper{suffix}_centrals_data_all.fits')

central_members.write(redmapper_centrals_path, overwrite=True)

# Consolidate Matched data

In [ ]:
redmapper_centrals_path = os.path.join(base_path, f'chunhao-redmapper{suffix}_centrals_data_all.fits')
central_members = Table.read(redmapper_centrals_path)
consolidate_hdf5_files_with_centrals(os.path.join(base_path, f'cluster{suffix}_data'), output_file, central_members)



# Check for missed members in which clusters (if there's a bad run)

In [7]:
import numpy as np
import pandas as pd
from astropy.table import Table

# Assuming redmapper_member_data_old and redmapper_member_data are Astropy Tables
old_cids = redmapper_member_data_old['mem_match_id'].data.astype(np.int32)
new_cids = redmapper_member_data['mem_match_id'].data.astype(np.int32)

# Count occurrences of mem_match_id in both arrays
old_counts = np.unique(old_cids, return_counts=True)
new_counts = np.unique(new_cids, return_counts=True)

# Create DataFrames from counts
old_counts_df = pd.DataFrame({'mem_match_id': old_counts[0], 'old_count': old_counts[1]})
new_counts_df = pd.DataFrame({'mem_match_id': new_counts[0], 'new_count': new_counts[1]})

# Merge the counts on mem_match_id
merged_counts = pd.merge(old_counts_df, new_counts_df, on='mem_match_id', how='outer').fillna(0)

# Find mismatches
mismatched_ids = merged_counts[merged_counts['old_count'] != merged_counts['new_count']]

# Output the mismatched cluster IDs
print("Mismatched cluster IDs:", mismatched_ids['mem_match_id'].tolist())


Mismatched cluster IDs: [67385, 163926, 411790, 416391, 453733, 461523, 479073, 801366, 820198]


In [10]:
np.savez(os.path.join(base_path, f'mismatched_clusters{suffix}.npz'),mismatched_ids=mismatched_ids)

In [ ]:
from astropy.table import Table, Column
from astropy.io import fits

halo_paths = ['/Users/shuleicao/Documents/halo_data_all.fits','/Users/shuleicao/Documents/halo_data_all_new.fits']

def get_name(file_path, names, mask=None, file_format='fits', key='gold'):
    if file_format == 'fits':
        table_data = Table.read(file_path)  # Directly read the FITS file into an Astropy Table
        if mask is not None:
            table_data = table_data[mask]  # Apply the mask directly to the table

        data_names = {name: table_data[name] for name in names}  # Extract the relevant columns
    elif file_format == 'hdf5':
        with h5py.File(file_path, 'r') as f:
            data_names = {name: f['catalog/'][key][name][:][mask] if mask is not None else f['catalog/'][key][name][:] for name in names}
    else:
        raise ValueError(f"Unknown file format: {file_format}")
    return data_names

def get_data(file_path, mask=None, file_format='fits', key='gold'):
    data_names = {}
    
    if file_format == 'fits':
        with fits.open(file_path) as hdul:
            data = hdul[1].data
            for name in data.names:
                data_names[name] = data[name][mask] if mask is not None else data[name]
    elif file_format == 'hdf5':
        with h5py.File(file_path, 'r') as f:
            for name in f['catalog/'][key].keys():
                data_names[name] = f['catalog/'][key][name][:][mask] if mask is not None else f['catalog/'][key][name][:]
    else:
        raise ValueError(f"Unknown file format: {file_format}")
    
    return data_names

# halo_data = get_data(halo_paths[0], file_format='fits')
halo_data_new = get_data(halo_paths[1], file_format='fits')


In [ ]:
import os
import h5py
import numpy as np

def read_all_clusters_hdf5_data(file_path):
    all_data = {}
    
    with h5py.File(file_path, 'r') as f:
        # Iterate through all clusters
        for cluster_id in f.keys():
            cluster_group = f.get(cluster_id)
            if cluster_group is None:
                continue

            cluster_data = {}
            for category in cluster_group.keys():
                if category != 'centrals':
                    category_group = cluster_group.get(category)
                    cluster_data[category] = []  
                    for halo_id in category_group.keys():
                        halo_group = category_group.get(halo_id)
                        if halo_group is not None:
                            halo_data = {}
                            halo_data['haloid'] = int(halo_id)
                            halo_data['m200'] = halo_group.attrs.get('m200', None)
                            halo_data['n_members'] = halo_group.attrs.get('n_members', None)
                            members_data = {}

                            # Read members data
                            for col_name in halo_group.keys():
                                members_data[col_name] = np.array(halo_group[col_name])

                            halo_data['members'] = members_data
                            cluster_data[category].append(halo_data)
                else:
                    category_data = cluster_group.get(category)
                    if category_data is not None:
                        members_data = {}

                        # Read members data
                        for col_name in category_data.keys():
                            members_data[col_name] = np.array(category_data[col_name])

                        cluster_data[category] = members_data
            
            all_data[cluster_id] = cluster_data

    return all_data
complete_data = read_all_clusters_hdf5_data(os.path.join(base_path, f'sorted-chunhao-redmapper{suffix}_data_new.h5'))


In [ ]:

def process_complete_data(complete_data, halo_data, redmapper_members):
    processed_data = {}


    for cluster_id, cluster_values in complete_data.items():
        largest_halo_data = {}
        central_matched_halo_data = {}

        mask = redmapper_members['mem_match_id']==int(cluster_id)
        masked_redmapper_members = redmapper_members[mask]
        id_to_p = dict(zip(masked_redmapper_members['id'], masked_redmapper_members['p'] * masked_redmapper_members['pfree']))

        # Calculate 'p_mem' for each halo and sort them
#         for halo in cluster_values.get('associated', []):
#             # Ensure 'coadd_object_id' is in the 'members' data
#             if 'coadd_object_id' in halo['members']:
#                 # Map each 'coadd_object_id' to its 'p' value, or 0 if not found
#                 halo['members']['p_mem'] = np.array(
#                     [id_to_p.get(coadd_id, 0) for coadd_id in halo['members']['coadd_object_id']]
#                 )
#                 # Calculate the sum of 'p_mem' for the current halo
#                 halo['p_mem_sum'] = np.sum(halo['members']['p_mem'])
#         sorted_halos = sorted(cluster_values.get('associated', []), key=lambda x: x.get('p_mem_sum', 0), reverse=True)

        for halo in cluster_values.get('associated', []):
            if 'coadd_object_id' in halo['members']:
                halo['members']['p_mem'] = np.array([id_to_p[coadd_id] for coadd_id in halo['members']['coadd_object_id'] if coadd_id in id_to_p])
                halo['p_mem_sum'] = np.sum(halo['members']['p_mem'])

        sorted_halos = sorted(cluster_values.get('associated', []), key=lambda x: x.get('p_mem_sum'), reverse=True)

#         if sorted_halos:
#             largest_halo = sorted_halos[0]
# #             print(f"Debug: Cluster ID {cluster_id}, Largest Halo ID: {largest_halo['haloid']}")

#             largest_halo_data['haloid'] = largest_halo['haloid']
# #             matching_indices = np.isin(largest_halo['members']['haloid'], largest_halo_data['haloid'])

# #             largest_halo_data['m200'] =  np.unique(largest_halo['members']['m200'][matching_indices])
#             largest_halo_data['m200'] = largest_halo['m200']
#             largest_halo_data['members'] = largest_halo['members']

#             # Check if the member haloids match
#             if any(largest_halo_data['haloid'] != member_halo_id for member_halo_id in largest_halo['members']['haloid']):
#                 print(f"Warning: Mismatch in haloid for cluster ID {cluster_id}")

#             # Extract "halo galaxies" from halo_data for the largest halo ID
#             halo_data_mask = (halo_data['haloid'] == largest_halo_data['haloid']) & (halo_data['m200'] == largest_halo_data['m200'])
#             halo_data_halo_galaxies = {key: halo_data[key][halo_data_mask] for key in halo_data.keys()}
#             largest_halo_data['halo_galaxies'] = halo_data_halo_galaxies
#         else:
#             # Handle the case where there are no associated halos
#             print(f"No associated halos for cluster ID {cluster_id}")
#             continue  # or set largest_halo_data to a default value

        if sorted_halos:
            halo_index = 0
            found_matching_halo = False

            while halo_index < len(sorted_halos) and not found_matching_halo:
                halo = sorted_halos[halo_index]
                halo_mask = (halo_data['haloid'] == halo['haloid']) & (halo_data['m200'] == halo['m200'])

                if np.any(halo_mask):
                    largest_halo_data['haloid'] = halo['haloid']
                    largest_halo_data['m200'] = halo['m200']
                    largest_halo_data['members'] = halo['members']

                    halo_data_halo_galaxies = {key: halo_data[key][halo_mask] for key in halo_data.keys()}
                    largest_halo_data['halo_galaxies'] = halo_data_halo_galaxies

                    found_matching_halo = True

                    # Check if the member haloids match
                    if any(largest_halo_data['haloid'] != member_halo_id for member_halo_id in halo['members']['haloid']):
                        print(f"Warning: Mismatch in haloid for cluster ID {cluster_id}")
                else:
                    halo_index += 1

            if not found_matching_halo:
                print(f"No matching halos found for cluster ID {cluster_id}")
        else:
            print(f"No associated halos for cluster ID {cluster_id}")

        # Match the haloid with centrals and save
        matched_central_data = None
        centrals_data = cluster_values.get('centrals', {})
        for i, central_haloid in enumerate(centrals_data.get('haloid', [])):
            if central_haloid in [halo['haloid'] for halo in sorted_halos] and centrals_data.get('rhalo', [])[i] == 0:
                matched_central_data = {col_name: centrals_data[col_name][i] for col_name in centrals_data.keys()}
                central_matched_halo_data['haloid'] = central_haloid
                central_matched_halo_data['members'] = next(halo for halo in sorted_halos if halo['haloid'] == central_haloid)['members']
                
                # Extract "halo galaxies" from halo_data for the central matched halo ID
                halo_data_mask = halo_data['haloid'] == central_matched_halo_data['haloid']
                halo_data_halo_galaxies = {key: halo_data[key][halo_data_mask] for key in halo_data.keys()}
                central_matched_halo_data['halo_galaxies'] = halo_data_halo_galaxies
                break

        # Fallback mechanism
        if matched_central_data is None and 'haloid' in centrals_data:
            matched_central_data = {}
            has_empty_arrays = False
            for col_name in centrals_data.keys():
                if len(centrals_data[col_name]) == 0:
                    if not has_empty_arrays:
                        print(f"Cluster with ID {cluster_id} has empty arrays in centrals data.")
                        has_empty_arrays = True
                    matched_central_data[col_name] = None
                else:
                    matched_central_data[col_name] = centrals_data[col_name][0]


        # Store in the processed data structure
        processed_data[cluster_id] = {
            'largest_halo': largest_halo_data,
            'central_matched_halo': central_matched_halo_data,
            'centrals': matched_central_data
        }
    
    return processed_data

processed_data = process_complete_data(complete_data, halo_data_new, redmapper_members)
# processed_data = process_complete_data(complete_data, masked_halo, redmapper_members)

def save_processed_data_to_hdf5(processed_data, output_path):
    with h5py.File(output_path, 'w') as f:
        for cluster_id, values in processed_data.items():
            cluster_group = f.create_group(cluster_id)

            # Function to save halo data
            def save_halo_data(group_name, halo_data):
                if 'halo_galaxies' not in halo_data:
                    return  # Skip saving if no halo_galaxies data is present

                halo_group = cluster_group.create_group(group_name)

                # Save halo_galaxies data
                halo_galaxies_group = halo_group.create_group('halo_galaxies')
                for col_name, col_data in halo_data['halo_galaxies'].items():
                    halo_galaxies_group.create_dataset(col_name, data=col_data)

                # Save members data
                members_group = halo_group.create_group('members')
                for col_name, col_data in halo_data['members'].items():
                    members_group.create_dataset(col_name, data=col_data)

            # Save data for largest_halo and central_matched_halo
            save_halo_data('largest_halo', values['largest_halo'])
            save_halo_data('central_matched_halo', values['central_matched_halo'])

            # Save centrals data
            centrals_group = cluster_group.create_group('centrals')
            for col_name, col_data in values['centrals'].items():
                try:
                    # Convert data to a numpy array for saving
                    data_to_save = np.array([col_data])
                    centrals_group.create_dataset(col_name, data=data_to_save)
                except TypeError:
                    # If the data type is unsupported, print out detailed information
                    print(f"Error with cluster_id: {cluster_id}, column: {col_name}, data: {col_data}")
                    # If the data is a string, encode it and save
                    if isinstance(col_data, str):
                        centrals_group.create_dataset(col_name, data=np.array([col_data.encode('utf-8')]), dtype=h5py.string_dtype(encoding='utf-8'))
                    # If the data is None, we won't save it
                    elif col_data is None:
                        continue
                    else:
                        # For other unsupported data types, you can add handling here
                        raise

    print("Data saved successfully!")

# Save the processed data
output_file_path = os.path.join(base_path, f'processed_sorted-chunhao-redmapper{suffix}_data_new_true_pmem.h5')
save_processed_data_to_hdf5(processed_data, output_file_path)


In [ ]:
import numpy as np
import os
import h5py
def read_processed_data_from_hdf5(input_path):
    processed_data = {}

    with h5py.File(input_path, 'r') as f:
        for cluster_id in f.keys():
            cluster_group = f.get(cluster_id)
            
            largest_halo_data = {}
            central_matched_halo_data = {}
            centrals_data = {}

            # Function to read halo data
            def read_halo_data(group_name):
                halo_data = {}
                
                halo_group = cluster_group.get(group_name)
                if halo_group is None:
                    return halo_data
                
                # Extract halo_galaxies data
                if 'halo_galaxies' in halo_group:
                    halo_galaxies_group = halo_group['halo_galaxies']
                    halo_data['halo_galaxies'] = {col_name: np.array(halo_galaxies_group[col_name]) for col_name in halo_galaxies_group.keys()}
                
                # Extract members data
                members_group = halo_group['members']
                halo_data['members'] = {col_name: np.array(members_group[col_name]) for col_name in members_group.keys()}
 
                # Extract haloid from the members data
                halo_data['haloid'] = halo_data['members']['haloid'][0]
                
                return halo_data

            # Read data for largest_halo and central_matched_halo
            largest_halo_data = read_halo_data('largest_halo')
            central_matched_halo_data = read_halo_data('central_matched_halo')

            # Read centrals data
            centrals_group = cluster_group.get('centrals')
            for col_name in centrals_group.keys():
                dataset = centrals_group[col_name]
                if dataset.dtype.kind == 'S':  # check if it's a bytestring
                    centrals_data[col_name] = dataset[0].decode('utf-8')
                else:
                    centrals_data[col_name] = dataset[0]

            # Store in the processed data structure
            processed_data[cluster_id] = {
                'largest_halo': largest_halo_data,
                'central_matched_halo': central_matched_halo_data,
                'centrals': centrals_data
            }

    return processed_data

# Read the processed data
input_file_path = os.path.join(base_path, f'processed_sorted-chunhao-redmapper{suffix}_data_new_true_pmem.h5')
loaded_data = read_processed_data_from_hdf5(input_file_path)
